In [1]:
import math

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy import stats

In [2]:
df = pd.read_csv(r"C:\Users\Lenovo\Desktop\Coursework\22_fall\ECON_309\Data\sof_data\survey_results_public.csv")
df.head()

,ResponseId,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,...,TimeSearching,TimeAnswering,Onboarding,ProfessionalTech,TrueFalse_1,TrueFalse_2,TrueFalse_3,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,None of these,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,"Employed, full-time",Fully remote,Hobby;Contribute to open-source projects,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Difficult,NaN
2,3,"I am not primarily a developer, but I write co...","Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Friend or family member...,Technical documentation;Blogs;Programming Game...,NaN,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Neither easy nor difficult,40205.0
3,4,I am a developer by profession,"Employed, full-time",Fully remote,I don’t code outside of work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;School (i.e., Universit...",NaN,NaN,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,215232.0
4,5,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Stack Overflow;O...,NaN,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN


In [3]:
professional_df = df[df['MainBranch']==df['MainBranch'][1]] # those who are only "I am a developer by profession"
professional_df = professional_df[professional_df['Employment']==professional_df['Employment'][1]] # those who are only full-time employyes
us_prof_df = professional_df[professional_df['Country'] == "United States of America"] # only USA, to deal with minimum wage/living cost differences around the world. 

In [4]:
 #getting rid of questions that are not needed for the model. 
us_prof_trimmed_df = us_prof_df.drop(
    columns=[
        "LanguageWantToWorkWith",
        "DatabaseWantToWorkWith",
        "PlatformWantToWorkWith",
        "WebframeWantToWorkWith",
        "MiscTechWantToWorkWith",
        "ToolsTechWantToWorkWith",
        "NEWCollabToolsWantToWorkWith",
        "TrueFalse_1",
        "TrueFalse_2",
        "TrueFalse_3",
        "SurveyLength",
        "SurveyEase",
        "Knowledge_1",
        "Knowledge_2",
        "Knowledge_3",
        "Knowledge_4",
        "Knowledge_5",
        "Knowledge_6",
        "Knowledge_7",
        "Frequency_1",
        "Frequency_2",
        "Frequency_3",
        "PurchaseInfluence",
        "BuyNewTool",
        "OpSysProfessional use",
        "OpSysPersonal use",
        "VersionControlSystem",
        "VCInteraction",
        "VCHostingPersonal use",
        "VCHostingProfessional use",
        "OfficeStackAsyncHaveWorkedWith",
        "OfficeStackAsyncWantToWorkWith",
        "OfficeStackSyncHaveWorkedWith",
        "OfficeStackSyncWantToWorkWith",
        "Blockchain",
        "NEWSOSites",
        "LearnCodeCoursesCert",
        "SOAccount",
        "SOComm",
        "Trans",
        "Sexuality",
        "Ethnicity",
        "Accessibility",
        "MentalHealth",
        "TBranch",
        "ICorPM",
    ]
)

In [5]:
us_prof_trimmed_df = us_prof_trimmed_df.fillna("***") # filling na values with a distinct string
us_prof_comp_data = us_prof_trimmed_df[~((us_prof_trimmed_df['CompTotal']=="***")&(us_prof_trimmed_df['ConvertedCompYearly']=="***"))] #getting rid of rows with missing dependent variable

In [6]:
us_prof_comp_data = us_prof_comp_data[us_prof_comp_data['ConvertedCompYearly']!="***"] 
us_prof_comp_data = us_prof_comp_data.drop(columns= ['CompTotal','CompFreq']) 
us_prof_comp_data

,ResponseId,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,YearsCode,YearsCodePro,...,SOVisitFreq,SOPartFreq,Age,Gender,WorkExp,TimeSearching,TimeAnswering,Onboarding,ProfessionalTech,ConvertedCompYearly
12,13,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","School (i.e., University, College, etc)",***,12,5,...,Daily or almost daily,***,25-34 years old,Man,5.0,30-60 minutes a day,Less than 15 minutes a day,Somewhat short,DevOps function;Microservices,65000.0
26,27,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",I don’t code outside of work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Other online resources ...,Technical documentation;Stack Overflow;Online ...,10,5,...,Daily or almost daily,A few times per month or weekly,25-34 years old,Man,***,***,***,***,***,106960.0
28,29,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Contribute to open-source projects,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Books / Physical media;School (i.e., Universit...",***,20,14,...,Multiple times per day,Multiple times per day,35-44 years old,Man,14.0,30-60 minutes a day,15-30 minutes a day,Somewhat long,Developer portal or other central places to fi...,130000.0
31,32,I am a developer by profession,"Employed, full-time",Fully remote,I don’t code outside of work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Other online resources ...,Technical documentation;Blogs;Stack Overflow;O...,24,21,...,Multiple times per day,A few times per week,55-64 years old,Man,21.0,30-60 minutes a day,15-30 minutes a day,Very long,DevOps function,102000.0
40,41,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Freelance/contract work,"Associate degree (A.A., A.S., etc.)",Books / Physical media;Other online resources ...,Technical documentation;Blogs;Written Tutorial...,20,15,...,A few times per week,A few times per month or weekly,35-44 years old,Man,***,***,***,***,***,130000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73105,73106,I am a developer by profession,"Employed, full-time",Fully remote,Hobby,Some college/university study without earning ...,"School (i.e., University, College, etc);On the...",***,9,7,...,A few times per week,Less than once per month or monthly,25-34 years old,Man,7.0,30-60 minutes a day,30-60 minutes a day,Somewhat long,DevOps function;Microservices;Continuous integ...,120000.0
73111,73112,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;School or academic work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Friend or family member;On the job training;On...,***,9,5,...,Daily or almost daily,***,25-34 years old,Man,***,***,***,***,***,115000.0
73113,73114,I am a developer by profession,"Employed, full-time",Fully remote,Hobby;Contribute to open-source projects,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Other online resources ...,Technical documentation;Blogs;Stack Overflow;O...,7,5,...,Daily or almost daily,I have never participated in Q&A on Stack Over...,25-34 years old,Man,7.0,15-30 minutes a day,Less than 15 minutes a day,Just right,DevOps function;Microservices;Developer portal...,94000.0
73116,73117,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Contribute to open-source projects;Freel...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Other online resources ...,Technical documentation;Written Tutorials,21,16,...,A few times per week,A few times per month or weekly,35-44 years old,Man,16.0,30-60 minutes a day,Less than 15 minutes a day,Very short,DevOps function;Microservices,115000.0


In [7]:
e_level =us_prof_comp_data['EdLevel'].unique().tolist() # coverting education level
e_level_list = [16, 18, 14, 13, 12, 22, 22, 12, 6] # corresponnding education years 
e_level_dict = dict(zip(e_level, e_level_list)) # create a dictionary
e_level_dict

{'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 16,
 'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 18,
 'Associate degree (A.A., A.S., etc.)': 14,
 'Some college/university study without earning a degree': 13,
 'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 12,
 'Other doctoral degree (Ph.D., Ed.D., etc.)': 22,
 'Professional degree (JD, MD, etc.)': 22,
 'Something else': 12,
 'Primary/elementary school': 6}

In [8]:
us_prof_comp_data['EdLevel_converted']= us_prof_comp_data['EdLevel'].apply(lambda x: e_level_dict.get(x)) # creating a new column of converted education years.

In [9]:
us_prof_comp_data = us_prof_comp_data[us_prof_comp_data['YearsCodePro'] != "***"] # getting rid of those who did not specificy 

In [10]:
cols_to_change = ['LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith']
for col in cols_to_change:
    new_col_name = col[:col.find("HaveWorked")]+"Count"
    us_prof_comp_data[new_col_name] = us_prof_comp_data[col].apply(lambda x: 0 if x =="***" else len(x.split(";"))) # parsing the list of programming languages/platforms/webframes etc, to get a count 
us_prof_comp_data['framelibsCount'] = us_prof_comp_data['PlatformCount'] +us_prof_comp_data['WebframeCount']+us_prof_comp_data['MiscTechCount']+us_prof_comp_data['ToolsTechCount']+us_prof_comp_data['NEWCollabToolsCount'] #com,bining "less" important techonological skills

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14492\2593767936.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_prof_comp_data[new_col_name] = us_prof_comp_data[col].apply(lambda x: 0 if x =="***" else len(x.split(";"))) # parsing the list of programming languages/platforms/webframes etc, to get a count
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14492\2593767936.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_prof_comp_data[new_col_name] = us_prof_comp_data[col].apply(lambda x: 0 if x =="***" 

In [11]:
us_prof_comp_data['YearsCodeProAdjusted'] = [0.5 if val == "Less than 1 year" else 55.0 if val == "More than 50 years" else float(val) for val in us_prof_comp_data['YearsCodePro']]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14492\2124214957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_prof_comp_data['YearsCodeProAdjusted'] = [0.5 if val == "Less than 1 year" else 55.0 if val == "More than 50 years" else float(val) for val in us_prof_comp_data['YearsCodePro']]


In [12]:
us_prof_comp_data['ConvertedCompYearly']=us_prof_comp_data['ConvertedCompYearly'].astype('float64')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14492\1026495942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_prof_comp_data['ConvertedCompYearly']=us_prof_comp_data['ConvertedCompYearly'].astype('float64')


In [13]:
set(us_prof_comp_data['OrgSize'])

{'***',
 '1,000 to 4,999 employees',
 '10 to 19 employees',
 '10,000 or more employees',
 '100 to 499 employees',
 '2 to 9 employees',
 '20 to 99 employees',
 '5,000 to 9,999 employees',
 '500 to 999 employees',
 'I don’t know',
 'Just me - I am a freelancer, sole proprietor, etc.'}

In [14]:
new_orgsize = []
for orgsize in us_prof_comp_data['OrgSize']:
    if orgsize == "***" or orgsize == "I don’t know" or orgsize == "Just me - I am a freelancer, sole proprietor, etc.":
        size == "***" # un-necessary data -> same as na values
    else:
        split_space = orgsize.split(" ") # splitting by space -> returns a list of strings
        min_size = split_space[0] # getting the 1st index, the 2, 10, 20, 100, and so on
        if orgsize == "10,000 or more employees":
            size = (10_000+ 20_000 )/2
        else:
            max_size = split_space[2] # getting the 2nd index, the 9, 19, 99, 499, and so on
            if min_size.find(",") != -1:
                min_size = min_size.replace(",", "") # removing the comma to convert to int
            if max_size.find(",") != -1:
                max_size = max_size.replace(",", "") # removing the comma to conv
            min_size = int(min_size)
            max_size = int(max_size)
            size = (min_size+max_size)/2
    new_orgsize.append(int(size))
us_prof_comp_data['NewOrgSize'] = new_orgsize

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14492\943164495.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_prof_comp_data['NewOrgSize'] = new_orgsize


In [15]:
print(set(us_prof_comp_data['LanguageCount'])) # clearly no one can be a professional developer without knowing any coding languages, this is most likely a survey error so we will remove these values. 
us_prof_comp_data = us_prof_comp_data[us_prof_comp_data['LanguageCount'] != 0] 

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 27}


In [16]:
for x  in us_prof_comp_data['ConvertedCompYearly']:
    if len(str(int(x))) <5: # check those who make less than 5 figure salary.
        print(int(x))

145
260
94
215
4260
140
180
5000
90
220
5232
6000
381
260
150
290
6000
9500
140
210
135
49
155
230
300


In [17]:
# these values seem to be outliers / errors from the survey 
# as any professional min wage job would make around $15,000
# so we wll remove them by converting them to 0s and removing the 0 rows.

us_prof_comp_data['ConvertedCompYearly'] = us_prof_comp_data['ConvertedCompYearly'].apply(lambda x: 0 if len(str(int(x))) <5 else x)
us_prof_comp_data= us_prof_comp_data[us_prof_comp_data['ConvertedCompYearly']!=0] 
# removing outliers in the dependent variable.
us_prof_comp_data= us_prof_comp_data[(np.abs(stats.zscore(us_prof_comp_data['ConvertedCompYearly'])) < 3)]
# https://www.bls.gov/oes/current/oes151251.htm considering the table of Percentile wage estimates for Computer Programmers from this website
# we can see that the 10% = $ 47,560 and the 25% $ 62,840. 
# Although they haven't provided us with an exact minimum, 
# it would be safe to assume that anything below $30,000 can be logically considered as an outlier. 
us_prof_comp_data = us_prof_comp_data[us_prof_comp_data['ConvertedCompYearly'] >= 30000.0]

In [18]:
us_prof_comp_data_mod1 = us_prof_comp_data[['ConvertedCompYearly', 'EdLevel_converted', 'YearsCodeProAdjusted', 'LanguageCount', 'framelibsCount', 'NewOrgSize']]

In [19]:
us_prof_comp_data_mod1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6943 entries, 12 to 73119
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ConvertedCompYearly   6943 non-null   float64
 1   EdLevel_converted     6943 non-null   int64  
 2   YearsCodeProAdjusted  6943 non-null   float64
 3   LanguageCount         6943 non-null   int64  
 4   framelibsCount        6943 non-null   int64  
 5   NewOrgSize            6943 non-null   int64  
dtypes: float64(2), int64(4)
memory usage: 379.7 KB


In [24]:
us_prof_comp_data_mod1.describe().to_excel(r'C:\Users\Lenovo\Desktop\college_work\ECON320_lab\03_datasets\d.xlsx')

In [20]:
import os
us_prof_comp_data_mod1.to_excel(os.path.join(r'C:\Users\Lenovo\Desktop\college_work\ECON320_lab\03_datasets',r'outlier_removed_orgsize_cleaned_data_stack_overflow_22_econ320.xlsx'), index= False)

In [21]:
reg = smf.ols(formula= 'np.log(ConvertedCompYearly) ~ EdLevel_converted + YearsCodeProAdjusted + LanguageCount + framelibsCount + NewOrgSize', data= us_prof_comp_data)
result = reg.fit()

print(f'Regression results model 1: \n{result}\n')
result.summary()

Regression results model 1: 



<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     np.log(ConvertedCompYearly)   R-squared:                       0.057
Model:                                     OLS   Adj. R-squared:                  0.056
Method:                          Least Squares   F-statistic:                     83.61
Date:                         Mon, 24 Oct 2022   Prob (F-statistic):           1.51e-85
Time:                                 16:30:42   Log-Likelihood:                -6572.6
No. Observations:                         6943   AIC:                         1.316e+04
Df Residuals:                             6937   BIC:                         1.320e+04
Df Model:                                    5                                         
Covariance Type:                     nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               11.2396      0.067    168.654      0.000      11.109      11.370
EdLevel_converted        0.0267      0.004      6.896      0.000       0.019       0.034
YearsCodeProAdjusted     0.0130      0.001     16.219      0.000       0.011       0.015
LanguageCount           -0.0022      0.004     -0.604      0.546      -0.009       0.005
framelibsCount           0.0086      0.002      5.196      0.000       0.005       0.012
NewOrgSize            1.156e-05   1.27e-06      9.127      0.000    9.08e-06     1.4e-05
==============================================================================
Omnibus:                     3251.038   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21254.382
Skew:                           2.160   Prob(JB):                         0.00
Kurtosis:                      10.404   Cond. No.                     6.56e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.56e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
reg = smf.ols(formula= 'np.log(ConvertedCompYearly) ~ YearsCodeProAdjusted + NewOrgSize', data= us_prof_comp_data)
result = reg.fit()

print(f'Regression results model 1: \n{result}\n')
result.summary()

In [ ]:
us_prof_comp_data_mod1.to_stata(os.path.join(r'C:\Users\Lenovo\Desktop\Coursework\22_fall\ECON_309\Data\sof_data',r'cleaned_data_stack_overflow_22_econ320.dta'))

In [ ]:
df = pd.read_stata(os.path.join(r'C:\Users\Lenovo\Desktop\Coursework\22_fall\ECON_309\Data\sof_data',r'cleaned_data_stack_overflow_22_econ320.dta'))

In [ ]:
df

In [ ]:
reg = smf.ols(formula= 'np.log(ConvertedCompYearly) ~ YearsCodeProAdjusted + NewOrgSize', data= df)
result = reg.fit()

print(f'Regression results model 1: \n{result}\n')
result.summary()

In [ ]:
df2= df.iloc[:,1:]
df2

In [ ]:

df2[(np.abs(stats.zscore(df2[df2.columns[0]])) < 3)].describe()

In [ ]:
df.describe().apply(lambda s: s.apply('{0:.5f}'.format))

In [ ]:
sorted(df['ConvertedCompYearly'].tolist())